## First move all pickles files to a source directory for batch processing

In [ ]:
import os
import shutil
from pathlib import Path

def copy_values_files(source_dir, destination_dir):
    """
    Recursively search for all files named "Values.pkl" in subdirectories of source_dir
    and copy them to the destination directory.
    
    Args:
        source_dir (str): The directory to search for "Values.pkl" files.
        destination_dir (str): The directory where the files will be copied.
    """
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Walk through the directory tree
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            # Check if the file is named "Values.pkl"
            if file == "Values.pkl":
                # Get the full path of the file
                file_path = os.path.join(root, file)
                # get the subdirectory strings to make mirrored folders
                file_name = os.path.normpath(file_path).split(os.path.sep)
                file_folder = os.path.join(*file_name[4:-1])
                
                # Construct the final destination path
                destination_path = os.path.join(destination_dir, file_folder, file)
                
                # make mirrored folders if they do not exist
                Path(os.path.join(destination_dir, file_folder)).mkdir(parents=True,exist_ok=True)
                # Copy the file to the destination directory
                shutil.copyfile(file_path, destination_path)
                print(f"Copied {file_path} to {destination_path}")

# call function
source_directory = "/Volumes/TOSHIBA/IMAGING"
destination_directory = "/Volumes/TOSHIBA/source_files"
copy_values_files(source_directory, destination_directory)

## Recursively open pkl files and save the data in a readable csv format

In [83]:
import pickle
import numpy as np
import scipy.stats as stats
import pandas as pd

source_dir = "/Volumes/TOSHIBA/source_files"

for root, dirs, files in os.walk(source_dir):
    for file in files:
        # Check if the file is named "Values.pkl"
        if file == "Values.pkl":
            # Get the full path of the file
            file_path = os.path.join(root,file)
            # load the pickle format
            try:
                raw,dF,dF_f0,mean_baseline,mean_dF_f0,__ = pickle.load(open(file_path, 'rb'))
            except:
                print(f"Cannot open {file_path}")
                continue

            # also output the mean and sem vector
            vars = [raw,dF,dF_f0]
            varnames = ['rawF','dF','dF_F0']
            varnames_mean = ['rawF_meanSEM','dF_meanSEM','dF_F0_meanSEM']

            # for each metric
            with pd.ExcelWriter('{}/summary_data.xlsx'.format(root)) as writer:
                for j,var in enumerate(vars):
                    # convert the variable to array
                    var = np.array(var,dtype='object')
                    df = pd.DataFrame(var)
                    df.to_excel(writer, sheet_name = varnames[j])

                    # then for each time point, compute the mean and sem over all larvae
                    mean = []
                    sem = []
                    for t in range(len(var[0])-1):
                        mean.append(np.mean([var[l][t] for l in range(len(var))]))
                        sem.append(stats.sem([var[l][t] for l in range(len(var))]))
                    df = pd.DataFrame({'mean':mean,'sem':sem})
                    df.to_excel(writer, sheet_name = varnames_mean[j])

                    summary = pd.Series([[np.mean(mean_baseline),stats.sem(mean_baseline)],[np.mean(mean_dF_f0),stats.sem(mean_dF_f0)]], index=['average','sem'])
                    df = pd.DataFrame({'mean baseline F0 per trial':mean_baseline,'mean dF_F0 per trial':mean_dF_f0})
                    df = pd.concat([df,summary])
                    df.to_excel(writer, sheet_name = 'summary')

Cannot open /Volumes/TOSHIBA/source_files/GCaMP_data/binary/R61D08_38H09LexA_LexAopGCaMP7s/set0/Values.pkl
Cannot open /Volumes/TOSHIBA/source_files/GCaMP_data/binary/R61D08_UAS-Ab1-40_38H09LexA_LexAopGCaMP7s/set0/Values.pkl
Cannot open /Volumes/TOSHIBA/source_files/GCaMP_data/binary/R61D08_UAS-Ab1-42_38H09LexA_LexAopGCaMP7s/set0/Values.pkl
